# 📊 EDA Overview
Exploratory Data Analysis for the Favorita Grocery Sales dataset.

In [1]:
# ==============================================================
# 📦 Setup & Imports
# ==============================================================
import sys
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")  # global clean mode

# --------------------------------------------------------------
# 0) Bootstrap project root
# --------------------------------------------------------------
ROOT = Path().resolve()
PROJECT_ROOT = ROOT.parent if ROOT.name == "notebooks" else ROOT
sys.path.append(str(PROJECT_ROOT))


print(f"📦 Project root: {PROJECT_ROOT}")
import sys
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.io as pio
import yaml

from src.data_loader import DataLoader, ensure_dir

# Plotly global
pio.templates.default = "plotly_dark"
pio.defaults.default_width = 900
pio.defaults.default_height = 500
pio.defaults.default_scale = 2

# Detect root
ROOT_CANDIDATE = Path().resolve()
PROJECT_ROOT = ROOT_CANDIDATE.parent if ROOT_CANDIDATE.name == "notebooks" else ROOT_CANDIDATE

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
CONFIG_PATH = PROJECT_ROOT / "configs" / "data" / "active.yaml"

print(f"📂 Project root: {PROJECT_ROOT}")
print(f"📁 Data dir:    {DATA_DIR}")
print(f"⚙️ Config path: {CONFIG_PATH}  (exists={CONFIG_PATH.exists()})")

loader = DataLoader(config_path=CONFIG_PATH)
print("✅ DataLoader initialized.")

📦 Project root: /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting
📂 Project root: /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting
📁 Data dir:    /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting/data
⚙️ Config path: /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting/configs/data/active.yaml  (exists=True)
✅ DataLoader initialized.


In [2]:
from src.utils.files import ensure_dirs
# Verify and prepare
IMG_DIR = PROJECT_ROOT / "img" / "reports" / "eda_overview"
ensure_dirs(IMG_DIR)

PosixPath('/Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting/img/reports/eda_overview')

## 📥 Load Train

In [3]:
print("📥 Loading train dataset…")
train = loader.load_dataset("train")
print(f"✅ Loaded train: {len(train):,} rows × {len(train.columns)} columns")
train.head()

📥 Loading train dataset…
📦 Found 5 parts for train. Merging…
✅ Loaded 103,857,647 rows × 6 cols.
✅ Loaded train: 103,857,647 rows × 6 columns


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25.0,103665.0,7.0,<NA>
1,1,2013-01-01,25.0,105574.0,1.0,<NA>
2,2,2013-01-01,25.0,105575.0,2.0,<NA>
3,3,2013-01-01,25.0,108079.0,1.0,<NA>
4,4,2013-01-01,25.0,108701.0,1.0,<NA>


## 📅 Date Range & Summary

In [4]:
print("📅 Date Range:")
if "date" in train.columns:
    print(train["date"].min(), "→", train["date"].max())

print("\n🔢 Basic Summary:")
train.describe(include="all")

📅 Date Range:
2013-01-01 00:00:00 → 2017-01-22 00:00:00

🔢 Basic Summary:


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.038576e+08,103857647,1.038576e+08,1.038576e+08,1.038576e+08,82199995
unique,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,76874911
mean,5.192882e+07,2015-05-14 06:02:55.569385728,2.735018e+01,9.333990e+05,8.647428e+00,NaN
min,0.000000e+00,2013-01-01 00:00:00,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,2.596441e+07,2014-06-25 00:00:00,1.200000e+01,5.113940e+05,2.000000e+00,NaN
50%,5.192882e+07,2015-07-21 00:00:00,2.800000e+01,9.363410e+05,4.000000e+00,NaN
75%,7.789323e+07,2016-05-01 00:00:00,4.300000e+01,1.260242e+06,9.000000e+00,NaN
max,1.038576e+08,2017-01-22 00:00:00,5.400000e+01,2.124052e+06,8.944000e+04,NaN


## 🕳️ Missing Values

In [5]:
na = train.isna().sum()
na = na[na > 0].sort_values(ascending=False)
na

onpromotion    21657652
store_nbr             1
item_nbr              1
unit_sales            1
dtype: int64

## 🏪 Store & Item Count

In [6]:
stores = train["store_nbr"].nunique()
items  = train["item_nbr"].nunique()

print(f"🏪 Unique Stores: {stores}")
print(f"📦 Unique Items:  {items}")

🏪 Unique Stores: 53
📦 Unique Items:  3900


## 📦 Store Distribution

In [7]:
store_counts = train["store_nbr"].value_counts().sort_index()

fig = px.bar(
    x=store_counts.index,
    y=store_counts.values,
    labels={"x": "Store Number", "y": "Count"},
    title="Store Distribution (Aggregated)"
)
fig.show()
fig.write_image(str(IMG_DIR / 'stores_distribution.png'))

## 📦 Item Distribution

In [18]:
# --- Top 30 items with metadata mapping (items.csv) ---

top_n = 30

# 1) Top-Items nach Anzahl der Zeilen im train-Set
item_counts = (
    train["item_nbr"]
    .value_counts()
    .rename_axis("item_nbr")
    .reset_index(name="count")
)

top_items = item_counts.head(top_n).copy()

# 2) Items-Metadaten laden
items = loader.load_dataset("items")
items_meta = items[["item_nbr", "family", "class", "perishable"]].drop_duplicates()

# 3) Join: Top-Items + Meta
top_items = top_items.merge(items_meta, on="item_nbr", how="left")

# 4) Label-Text für Y-Achse (Family + ID)
top_items["label"] = top_items.apply(
    lambda row: f"{row['family']} (#{int(row['item_nbr'])})"
    if pd.notna(row["family"])
    else f"Item #{int(row['item_nbr'])}",
    axis=1,
)

# 5) Sortieren für horizontales Bar-Chart
top_items = top_items.sort_values("count", ascending=True)

# Reihenfolge für die Achse fixieren
label_order = top_items["label"].tolist()

# 6) Plot – nur links Labels, keine Text-Labels rechts
fig = px.bar(
    top_items,
    x="count",
    y="label",
    orientation="h",
    title=f"Top {top_n} Most Frequent Items (with Families)",
    labels={
        "count": "Number of Sales Records",
        "label": "Item (Family + ID)",
    },
    hover_data=["item_nbr", "family", "class", "perishable"],
)

# Y-Achse: alle Labels in exakt dieser Reihenfolge anzeigen
fig.update_yaxes(
    categoryorder="array",
    categoryarray=label_order,
    automargin=True,
)

# Plot höher machen, damit wirklich JEDER Balken sein Label bekommt
bar_height = max(700, top_n * 28)  # 28 px pro Balken, min. 700 px
fig.update_layout(
    margin=dict(l=260, r=40, t=60, b=40),
    height=bar_height,
)

fig.show()
fig.write_image(str(IMG_DIR / "top_30_items.png"))
print("✅ Saved top_30_items.png with all labels on the left.")

📦 Found 1 parts for items. Merging…
✅ Loaded 4,100 rows × 4 cols.


✅ Saved top_30_items.png with all labels on the left.


## 📈 Unit Sales Distribution

In [9]:
item_counts = (
    train["item_nbr"]
    .value_counts()
    .rename_axis("item_nbr")
    .reset_index(name="n_rows")
)

fig = px.histogram(
    item_counts,
    x="n_rows",
    nbins=60,
    title="Distribution of Sales Records per Item",
    labels={"n_rows": "Number of Rows (Sales Records)"},
)
fig.update_layout(bargap=0.05)
fig.show()
fig.write_image(str(IMG_DIR / 'unit_sales_distribution.png'))

# 🗓️ Sales by day of week

In [10]:
train['day_of_week'] = pd.to_datetime(train['date']).dt.day_name()
dow = train.groupby('day_of_week')['unit_sales'].mean().reindex([
    'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'
]).reset_index()
fig = px.bar(dow, x='day_of_week', y='unit_sales', title='Average Sales by Day of Week', template='plotly_dark')
fig.show()
fig.write_image(str(IMG_DIR / 'avg_sales_by_dayofweek.png'))

# 🏪 Store Performance

In [11]:
store_perf = train.groupby('store_nbr')['unit_sales'].mean().reset_index().sort_values('unit_sales', ascending=False)
fig = px.bar(store_perf.head(30), x='store_nbr', y='unit_sales', title='Top 30 Stores by Average Sales', template='plotly_dark')
fig.show()
fig.write_image(str(IMG_DIR / 'top_stores_avg_sales.png'))

# 🔥 Promotions & Sales

In [12]:
promo_df = train.groupby('onpromotion')['unit_sales'].mean().reset_index()
fig = px.bar(promo_df, x='onpromotion', y='unit_sales', title='Average Sales vs Promotion Status', template='plotly_dark')
fig.show()
fig.write_image(str(IMG_DIR / 'promo_vs_sales.png'))

## 📈 Daily Total Sales Over Time

In [13]:
ts = train.groupby("date")["unit_sales"].sum().reset_index()

fig = px.line(ts, x="date", y="unit_sales", title="Daily Total Sales")
fig.show()
fig.write_image(str(IMG_DIR / 'total_sales_over_time.png'))